The first thing to do is to load our specimen into memory and work out the subslicing, we use the function  `generate_slicing_figure` to help us here. We want the red dashed lines to ideally sit just ontop of the atoms in the column. Aim for slices to be spaced around 2 $\\AA$.

In [ ]:
import pyms
import numpy as np
%matplotlib inline
structure = pyms.structure.fromfile("Structures/SrTiO3_CeO2_interface.xyz",atomic_coordinates='cartesian')
# A few maniupulations to remove vaccuum at edges and create a psuedo-periodic
# structure
structure = structure.resize([0.1, 0.76], axis=0)
from copy import deepcopy
other = deepcopy(structure).resize([0.017, 0.99], axis=0)
other.reflect([0])
structure = structure.concatenate(other, axis=0)
# Uncomment this line to output a vesta viewable version of this structure
# structure.output_vesta_xtl('SrTiO3_CeO2_interface_for_vesta.xtl')


# Here is the array determining 
slice_size = 2/structure.unitcell[2]
slices = np.arange(0.02,1.0,slice_size)

_ = structure.generate_slicing_figure(slices,show=False)


Now we need to choose the appropriate tiling and grid sampling for the simulation.

A multislice calculation assumes periodic boundary conditions. To avoid artefacts associated with this the pixel grid must be chosen to have sufficient size so that the probe does not artificially interfere with itself through the periodic boundary (wrap around error). The grid sampling must also be sufficient that electrons scattered to high angles are not scattered beyond the band-width limit of the array.

The function `generate_probe_spread_plot` plot helps identify whenever these two events are happening. If the probe intensity drops below 0.95 (as a fraction of initial intensity) then the grid is not sampled finely enough, the pixel size of the array (gridshape) needs to increased for finer sampling of the specimen potential. If the probe spread exceeds 0.2 (as a fraction of the array) then too much of the probe is spreading to the edges of the array, the real space size of the array (usually controlled by the tiling of the unit cell) needs to be increased.

In [ ]:

import torch

# Inadequate pixel sampling - probe intensity will drop too much
gridshape = [256, 256]
tiling = [1, 7]

# Adequate real space sampling - probe will spread outside of simulation array
gridshape = [512, 512]
tiling = [1, 2]

# Adequate pixel sampling
#gridshape = [512, 512]
#tiling = [1, 4]

# The above results might change with these parameters, have a play around
# Probe accelerating voltage
eV = 3e5
app = 20
probe_posn = [0.5, 0.0]
df = 0
thickness = 200

_ = pyms.generate_probe_spread_plot(
    gridshape,
    structure,
    eV,
    app,
    thickness,
    subslices=slices,
    tiling=tiling,
    showcrossection=True,
    probe_posn=probe_posn,
    df = df,
    device=torch.device('cpu')
)